In [1]:
import os
from dotenv import load_dotenv
import openai
import json
import requests
import yaml
import copy

load_dotenv()

True

In [2]:
# function calling用関数のベースとなるクラス
class BaseClass:
    # 関数のメタデータのベース
    metadata_base = {
        "name": "",
        "description": "",
        "parameters": {
            "type": "object",
            "properties": {
                "url": {
                    "type": "string",
                    "description": "URL of the API endpoint",
                    "enum": []
                },
                "method": {
                    "type": "string",
                    "description": "HTTP method",
                    "enum": []
                },
                "body": {},
            },
            "required": [],
        },
    }

    def __init__(self):
        pass

    # ベースクラスに共通のrunメソッドを追加
    def run(url: str, method: str, body: dict = None) -> dict:
        if method == "GET":
            response = requests.get(url)
        elif method == "POST":
            response = requests.post(url, json=body)
        else:
            raise Exception("Not supported method")

        response_body = response.json()
        return response_body



# openapi.yamlを元にFunction calling用のメタデータを作成する関数
def create_function_metadata_from_openapi(
        endpoint: str, 
        method: str, 
        method_info: dict, 
        schemas: dict
    ) -> dict:

    # メタデータのベースをコピー
    metadata = dict(BaseClass.metadata_base)

    # メタデータ作成に必要なデータを取得
    name = method_info.get("operationId")
    description = method_info.get("summary")
    method = method.upper()
    if method == "POST":
        schema_ref = method_info.get("requestBody", {}).get("content", {}).get("application/json", {}).get("schema", {}).get("$ref", "")
        schema_name = schema_ref.split("/")[-1]
        body = schemas.get(schema_name, {})
        required_properties = ["url", "method", "body"]
    else:
        body = {}
        required_properties = ["url", "method"]

    # メタデータの作成
    metadata["name"] = name
    metadata["description"] = description
    metadata["parameters"]["properties"]["url"]["enum"] = [endpoint]
    metadata["parameters"]["properties"]["method"]["enum"] = [method]
    if body == {}: # bodyが空の場合は削除
        metadata["parameters"]["properties"].pop("body")
    else:
        metadata["parameters"]["properties"]["body"] = body
    metadata["parameters"]["required"] = required_properties


    return metadata

In [3]:
# Function calling用のクラスを格納する辞書
function_classes = {}


# openapi.yamlを指定したURLからダウンロード
base_url = "http://localhost:5000"
openapi_url = f"{base_url}/openapi.yaml"
response = requests.get(openapi_url)
openapi_yaml = response.text
openapi_data = yaml.safe_load(openapi_yaml)


# サーバーのURLを取得
servers = [server.get("url") for server in openapi_data.get("servers", [{}])]


# openapi.yamlを入力としてクラスを動的に生成
for path, methods in openapi_data.get("paths", {}).items():
    for method, method_info in methods.items():
        metadata = create_function_metadata_from_openapi(
            endpoint=servers[0] + path, 
            method=method,
            method_info=method_info,
            schemas=openapi_data.get("components", {}).get("schemas", {}),
        )
        
        # Function calling用のクラスを個々に作成し、辞書に格納
        function_classes[metadata["name"]] = type(
            metadata["name"],
            (BaseClass,),
            {
                # copy()だと参照渡しになって上書きされてしまうので、deepcopy()を使用してコピー
                "metadata": copy.deepcopy(metadata),  
            },
        )


# Function callingで使用可能な状態にする
functions_metadata = [function_class.metadata for function_class in function_classes.values()]
functions_callable = {function_class.metadata["name"]: function_class.run for function_class in function_classes.values()}

functions_metadata

[{'name': 'getTodos',
  'description': 'Get the list of todos',
  'parameters': {'type': 'object',
   'properties': {'url': {'type': 'string',
     'description': 'URL of the API endpoint',
     'enum': ['http://localhost:5000/todos']},
    'method': {'type': 'string',
     'description': 'HTTP method',
     'enum': ['GET']}},
   'required': ['url', 'method']}},
 {'name': 'postTodo',
  'description': 'Add a todo to the list',
  'parameters': {'type': 'object',
   'properties': {'url': {'type': 'string',
     'description': 'URL of the API endpoint',
     'enum': ['http://localhost:5000/todos']},
    'method': {'type': 'string',
     'description': 'HTTP method',
     'enum': ['POST']},
    'body': {'properties': {'todo': {'description': 'The todo to add',
       'type': 'string'}},
     'type': 'object'}},
   'required': ['url', 'method', 'body']}}]

In [8]:
# Azure OpenAIの設定
openai.api_type = "azure"
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")


# システムのプロンプト
SYSTEM_PROMPT = """
あなたはユーザを助けるアシスタントです。
ユーザの入力に正しく回答を出力するために、ステップバイステップで慎重に考えることができます。
まずはゴール達成のためになにが必要かを考え、自分の思考と行動を説明します。
"""

messages = [
    {"role":"system", "content": SYSTEM_PROMPT},
]


def exec_function_calling(user_input:str):
    # ユーザの入力をメッセージに追加
    messages.append({"role": "user", "content": user_input})

    # 推論実行
    response = openai.ChatCompletion.create(
        engine = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
        messages = messages,
        functions=functions_metadata,
        function_call="auto",
        temperature=0,
    )

    # 関数の呼び出し有無を確認
    while response["choices"][0]["message"].get("function_call"):
        msg = response["choices"][0]["message"]
        func_name = msg["function_call"]["name"]
        print(msg["function_call"]["arguments"])
        func_args = json.loads(msg["function_call"]["arguments"])

        # 関数を呼び出し
        print(f"関数名： {func_name}")
        print(f"引数 ： {func_args}\n\n")
        func_result = functions_callable[func_name](**func_args)
        
        # 関数の実行結果をメッセージに追加
        status_msg = "関数:{}を実行\n実行結果:{}".format(func_name, func_result)
        messages.append(
            {
                "role": "function",
                "name": func_name, 
                "content": status_msg
            }
        )

        # 再度、推論実行
        response = openai.ChatCompletion.create(
            engine = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
            messages = messages,
            functions=functions_metadata,
            function_call="auto"
        )

    # 結果をメッセージに追加する
    result = response["choices"][0]["message"]["content"]
    messages.append({"role": "assistant", "content": result})    
    
    return result

In [9]:
# Function callingの実行
user_input = "Todoリストに「Qiitaに記事を書く」がない場合は追加したい"
result = exec_function_calling(user_input)
print(result)

{
  "url": "http://localhost:5000/todos",
  "method": "GET"
}
関数名： getTodos
引数 ： {'url': 'http://localhost:5000/todos', 'method': 'GET'}


{
  "url": "http://localhost:5000/todos",
  "method": "POST",
  "body": {
    "todo": "Qiitaに記事を書く"
  }
}
関数名： postTodo
引数 ： {'url': 'http://localhost:5000/todos', 'method': 'POST', 'body': {'todo': 'Qiitaに記事を書く'}}


まず、Todoリストを取得するためにgetTodos関数を実行します。取得したリストは以下のようになっています。

- todo1
- todo2
- todo3

次に、取得したリストに「Qiitaに記事を書く」が含まれているかを確認します。現在のリストには含まれていないので、追加する必要があります。

「Qiitaに記事を書く」をTodoリストに追加するために、postTodo関数を実行します。実行後、Todoリストは以下のようになります。

- todo1
- todo2
- todo3
- Qiitaに記事を書く

以上で、Todoリストに「Qiitaに記事を書く」が追加されました。


In [10]:
# Function callingの実行
user_input = "お願いします"
result = exec_function_calling(user_input)
print(result)

{
  "url": "http://localhost:5000/todos",
  "method": "GET"
}
関数名： getTodos
引数 ： {'url': 'http://localhost:5000/todos', 'method': 'GET'}


Todoリストを確認したところ、「Qiitaに記事を書く」が既に追加されています。追加されているため、再度追加する必要はありません。現在のTodoリストは以下の通りです。

- todo1
- todo2
- todo3
- Qiitaに記事を書く

ご要望の「Qiitaに記事を書く」は既にTodoリストに含まれていますので、追加する必要はありません。
